### Pair Programming Asunciones Regresión Lineal


En este ejercicio de pair programming seguiréis usando el dataset que seleccionasteis.
El objetivo de este pairprgramming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. Recordamos que estas asunciones son:  

- Normalidad (ya la deberíais haber evaluado)

- Homogeneidad de varianzas.

- Independencia de las variables.

Cada asunción la deberéis testar de forma visual y analítica.

In [6]:
import numpy as np
import pandas as pd
import random 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,8)
import warnings
warnings.filterwarnings('ignore')

#### - Homogeneidad de varianzas.


#### - Independencia de las variables.
